In [1]:
import ee
import geemap
import os
import csv

In [2]:
Map =geemap.Map()
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [11]:
# imput edited mangrove 2018 field data
fieldData = r'C:\Users\sgao5\Downloads\OutputData\OutputData\MangroveHainan18Fielddata.shp'
# fieldData = geemap.shp_to_ee(polygon.shp)
# Map.addLayer(fieldData,{},'Mangrove Field data 2018')

# add mangrove core area
#corearea = '../Mangrove/CoreArea.shp'

# add Hainan province rectangle 111.087E, 18.122N. 108.455E, 20.076N
Hainan = ee.Geometry.Rectangle([111.087, 18.122, 108.455, 20.076])

# to mask clouds based on the pixel_qa band of Landsat 8 SR data 
def maskL8sr(image):
  # Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

# define image as landsat 8 SR dataset from May to August # Why use SR?
image = ee.ImageCollection('LANDSAT/LC08/C01/T2_SR') \
    .filterBounds(Hainan) \
    .filterDate('2018-05-01', '2018-8-31') \
    .map(maskL8sr)
image = image.mean()
# set bands for prediction
bands = ['B1','B2','B3','B4','B5','B6','B7'] # why bring B1 to this list?


In [15]:
# define training points
mangrove_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\mangrove18T.shp'
water_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\water18T'
forest_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\forest18T'
constructionLand_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\construct18T'
cultivatedLand_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\culti18T'
pond_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\pond18T'
# define validating points
mangrove_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\mangrove18V.shp"
water_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\water18V.shp"
forest_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\forest18V.shp"
constructionLand_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\construct18V.shp"
cultivatedLand_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\culti18V.shp"
pond_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\pond18V.shp"
# landcover label *
label = 'landcover'
# merge points
mgt_points = mangrove_t.merge(water_t).merge(forest_t).merge(cultivatedLand_t).merge(pond_t).merge(constructionLand_t)
mgv_points = mangrove_v.merge(water_v).merge(forest_v).merge(cultivatedLand_v).merge(pond_v).merge(constructionLand_v)
# Map.addLayer(mgpoints, {}, 'training points', False)
print(mgt_points, 'mgtpoints')
print(mgv_points, 'mgvpoints')

AttributeError: 'str' object has no attribute 'merge'

In [ ]:
# overlay the points on the imagery to get training
training = image.select(bands).sampleRegions(**{
    'collection' : mgt_points,
    'scale' : 30,
    'properties' : 'landcover',
    'projection' : 32649
})

validating = image.select(bands).sampleRegions(**{
    'collection' : mgv_points,
    'scale' : 30,
    'properties' : 'landcover'
    'projection' : 32649
})
# Adds a column of deterministic pseudorandom numbers/ split training and varification dataset
trainingSample = training.randomColumn()
validatingSample = validating.randomColumn()

training = trainingSample.filter(ee.Filter.lt('random', 1))
validation = validatingSample.filter(ee.Filter.lt('random', 1))

In [ ]:
# Train the classifier
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

# classify the image with the same bands used for training
result = image.select(bands).classify(classifier)
# display the clusters with given colors
vis_params = {
    'min' : 1,
    'max' : 6,
    'palette' :['98ff00','62ffde','0b4a8b','fffff0','0dd66b','ff78d2']
}
Map.addLayer(result, vis_params, 'classified')


In [ ]:
# add map elements
print('layer opacity:')
clusterLayer = Map.layers[-1]
clusterLayer.interact(opacity = (0,1,0.1))
# add legend
legend_dict = {
    '1 mangrove' : '98ff00',
    '2 water' : '62ffde',
    '3 forest' : '0b4a8b',
    '4 cultivated land' : 'fffff0',
    '5 pond' : '0dd66b',
    '6 construction land' : 'ff78d2'
}
Map.add_legend(legend_title = 'Hainan Mangrove Classification', legend_dict = legend_dict)
Map

In [ ]:
# Accuracy assessment
trainingAccuracy = classifier.confusionMatrix()
# validation dataset
validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('landcover', 'classification')

In [ ]:
# Download confusion matrix
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')
with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(trainingAccuracy.getInfo())
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testAccuracy.getInfo())
# reclassify landcover map ?

# export the raster
# out_file = os.path.join(out_dir, 'mangroveClassification.tif')
# geemap.ee_export_image(landcover, filename=out_file, scale=900)